<a href="https://colab.research.google.com/github/bblodfon/fixpoints_asp/blob/master/ClingoOnColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook introduces the use of [Google Colab](https://colab.research.google.com/) for sharing executable sketches of [ASP-based systems](https://arxiv.org/abs/2008.06692). This is useful when someone who *is* familiar with ASP want to communicate to someone who *is not* familiar with ASP how ASP might be used to solve their problem. In particular, it skips over most of the details of the problem formulation in ASP to give focus to concrete details about how the system might be deployed.

Some things demonstrated:
 - Installing `clingo`
 - Defining a simple and compact problem file format that you might share with collaborators (who will give you more realistic problem instances in the future)
 - Converting the problem format into a set of problem instance definition facts
 - Encoding a semantics of the problem in ASP
 - Executing `clingo` to get one or more solutions
 - Packaging overall system in a Python function

# Installing `clingo`

In [ ]:
!apt-get -qq install gringo

In [ ]:
!clingo --version

clingo version 5.2.2
Address model: 64-bit

libgringo version 5.2.2
Configuration: with Python 3.6.4, with Lua 5.3.3

libclasp version 3.3.3 (libpotassco version 1.0.1)
Configuration: WITH_THREADS=1
Copyright (C) Benjamin Kaufmann

License: The MIT License <https://opensource.org/licenses/MIT>


# Problem Background

Problem definition: [finding a fixpoint of a system of pseudo-boolean update rules](https://math.stackexchange.com/questions/3797239/solver-for-finding-fixpoints-of-a-boolean-system) as recently discussed on [`potassco-users` mailing list](https://sourceforge.net/p/potassco/mailman/potassco-users/?viewmonth=202008&viewday=21).

# Problem file format

The problem hinges on finding assignments to $n$ Boolean variables $x_i$ such that $x_i = [ \sum_j{a_{ij}x_j} ]$ (where $[]$ evaluates non-negativity). Given the matrix $a$, a problem instance is fully defined.

Because these matrices might get large,  we'll use a [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html) data structure to represent these matrices in memory. For storage and interchange, we'll let scipy use its own [compressed serialization format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.save_npz.html).

In [ ]:
import scipy.sparse

a = scipy.sparse.lil_matrix((3,3),dtype='int32')

# x_1 := [ -x_2 ]
a[0,1] = -1 

# x_2 := [ x_1 - x_3 ]
#a[1,0] = 1 # per discussion, this is commented out to allow the problem to be satisfiable
a[1,2] = -1

# x_3 := [ x_1 + x_3 ]
a[2,0] = 1
a[2,2] = 1

scipy.sparse.save_npz("example.npz", a.tocoo())

!ls -lh example.npz
# Space-savings from the file format choice are not visible on this tiny instance

-rw-r--r-- 1 root root 955 Aug 22 22:58 example.npz


# Converting problem file into ASP format

In [ ]:
def write_problem_instance(a,f):
  f.write(b"num_variables(%d).\n" % a.shape[0])
  for ijc in zip(a.row, a.col, a.data):
    f.write(b"coef(%d,%d,%d).\n" % ijc)

a = scipy.sparse.load_npz("example.npz")
with open("example.lp", "wb") as f:
  write_problem_instance(a, f)

In [ ]:
!cat example.lp

num_variables(3).
coef(0,1,-1).
coef(1,2,-1).
coef(2,0,1).
coef(2,2,1).


# Encoding semantics of the general problem

In [ ]:
%%file solution.lp

var(0..N-1) :- num_variables(N).
{ hot(I) } :- var(I).

 hot(I) :- var(I), #sum { C, J : coef(I, J, C), hot(J) } >= 0.
-hot(I) :- var(I), #sum { C, J : coef(I, J, C), hot(J) } < 0.

#show hot/1.

Overwriting solution.lp


# Executing Clingo to get some solutions

It's always a good idea to use `--text` on a small problem instance to make sure the grounder is expanding your first-order rules the way you expect.

In [ ]:
!clingo --text example.lp solution.lp

num_variables(3).
coef(2,0,1).
coef(2,2,1).
coef(0,1,-1).
coef(1,2,-1).
var(0).
var(1).
var(2).
{hot(0)}.
{hot(1)}.
{hot(2)}.
hot(2):-#delayed(1).
hot(1):-#delayed(2).
hot(0):-#delayed(3).
-hot(0):-0>#sum{-1,1:hot(1)}.
-hot(1):-0>#sum{-1,2:hot(2)}.
:-hot(0),-hot(0).
:-hot(1),-hot(1).
#delayed(1) <=> #sum{1,2:hot(2);1,0:hot(0)}
#delayed(2) <=> 0<=#sum{-1,2:hot(2)}
#delayed(3) <=> 0<=#sum{-1,1:hot(1)}
#show hot/1.


Just how many solutions are there to this tiny example problem? One.

In [ ]:
!clingo example.lp solution.lp 0

clingo version 5.2.2
Reading from example.lp ...
Solving...
Answer: 1
hot(2) hot(0)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


# Packaging overall system in a Python function

This is the kind of function you might use in some Python server software. Because it runs the solver in a subprocess and makes no writes to the disk (and has no other shared state), it is safe to call concurrently from different threads in a way that yields parallelism. Error handling (e.g. for when the solver is spontaneously killed for using all of the server's memory) could be improved, but at least it supports the SAT and UNSAT cases.

In [ ]:
import subprocess
import json

def pb_system_fixpoints(a, max_solutions=0):

  clingo = subprocess.Popen(
      ["/usr/bin/clingo","-", str(max_solutions), "--outf=2"],
      stdin=subprocess.PIPE,
      stdout=subprocess.PIPE)

  clingo.stdin.write(b"""
    var(0..N-1) :- num_variables(N).
    { hot(I) } :- var(I).

     hot(I) :- var(I), #sum { C, J : coef(I, J, C), hot(J) } >= 0.
    -hot(I) :- var(I), #sum { C, J : coef(I, J, C), hot(J) } < 0.

    #show hot/1.
  """)

  clingo.stdin.write(b"num_variables(%d).\n" % a.shape[0])

  for ijc in zip(a.row, a.col, a.data):
    clingo.stdin.write(b"coef(%d,%d,%d).\n" % ijc)

  clingo.stdin.close()

  result = json.load(clingo.stdout)
  
  clingo.wait()

  if result['Result'] == 'SATISFIABLE':
    witnesses = result['Call'][0]['Witnesses']
    fixpoints = scipy.sparse.lil_matrix((len(witnesses),a.shape[0]),dtype=bool)
    for i,w in enumerate(witnesses):
      for term in w['Value']:
        assert term.startswith('hot(')
        fixpoints[i,int(term[4:-1])] = 1
    return fixpoints.tocsc()
  else:
    return None

In [ ]:
pb_system_fixpoints(a).todense()

matrix([[ True, False,  True]])